todo: 
- plot loss. see if jumps up 
- drop out. (may increase interpretability of mid layers?)

In [ ]:
import os
from time import perf_counter, time
from functools import lru_cache
import pickle
import matplotlib.pylab as plt
import numpy as np
from scipy.ndimage import zoom
from IPython import display
from imageio import imread

In [ ]:
import tensorflow as tf 
from tensorflow.keras.layers import Dense, InputLayer
from tensorflow.keras import Model

print("TF version:", tf.__version__)
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

In [ ]:
try:
    from google.colab import drive
except ModuleNotFoundError:
    ROOT = 'images'
else:
    drive.mount('/content/drive')
    ROOT = '/content/drive/My Drive/Colab Notebooks/images'

In [ ]:
class MyModel(Model):
  def __init__(self, width, depth, n_channels = 3):
    super(MyModel, self).__init__()
    self.width = width
    self.depth = depth
    self.n_channels = n_channels
    self.myLayers = []
    for i in range(depth):
      layer = Dense(
        width, activation='relu', name = f'relu_layer_{i}', 
        kernel_initializer=tf.initializers.RandomNormal(
          stddev = (width** -.5), 
        ), 
        bias_initializer  =tf.initializers.RandomNormal(stddev=0.01),
      )
      self.myLayers.append(layer)
    self.last = Dense(n_channels, activation='sigmoid', name = f'sigmoid_layer')

  def call(self, x):
    for layer in self.myLayers:
      x = layer(x)
    return self.last(x)
  
  def build(self, shape = (None, 2)):
    super().build(shape)

  def copy(self):
    model = MyModel(self.width, self.depth, self.n_channels)
    model.build()
    model.set_weights(self.get_weights())
    return model


In [ ]:
@lru_cache()
def getRaster(width, height):
  buffer = np.zeros((width*height, 2))
  x_lin_space = np.linspace(-1, 1, width)
  y_lin_space = np.linspace(-1, 1, height)
  for x in range(width):
    for y in range(height):
      buffer[x * height + y, :] = (x_lin_space[x], y_lin_space[y])
  return buffer

In [ ]:
def view(model, width, height, view_h = 5):
    output = model.predict(getRaster(width, height))
    plt.imshow(
        np.reshape(output, (width, height, model.n_channels)), 
        vmin=0, vmax=1, 
    )
    plt.axis('off')
    plt.gcf().set_size_inches(view_h / height * width, view_h)

In [ ]:
def viewInitField():
    model = MyModel(4, 4, 3)
    model.build()
    model.summary()
    view(model)
# viewInitField()

In [ ]:
os.chdir(ROOT)
img_names = os.listdir()

In [ ]:
def loadData(img_name, resolution = 50, to_gray = False):
  img = imread(img_name) / 255
  width, height = img.shape[:2]
  try:
    if img.shape[2] == 4:
      img = img[:, :, :3]
    elif img.shape[2] == 1:
      raise IndexError
    else:
      assert img.shape[2] == 3
  except IndexError:
    t = np.zeros((width, height, 3))
    t[:, :, 0] = img
    t[:, :, 1] = img
    t[:, :, 2] = img
    img = t
  zoom_k = resolution / (width * height) ** .5
  tt = zoom(img[:, :, 0], zoom_k, order=1)
  width, height = tt.shape
  t = np.zeros((width, height, 3))
  t[:, :, 0] = tt
  t[:, :, 1] = zoom(img[:, :, 1], zoom_k, order=1)
  t[:, :, 2] = zoom(img[:, :, 2], zoom_k, order=1)
  img = t
  if to_gray:
    img[:, :, 0] = np.mean(img, axis=2)
    n_channels = 1
  else:
    n_channels = 3
  x = getRaster(width, height)
  y = np.zeros((width * height, n_channels))
  for i in range(n_channels):
    y[:, i] = np.reshape(img[:, :, i], (width * height, ))
  return x, y, width, height, img

In [ ]:
previewIter = iter(img_names)

In [ ]:
# Run this cell multiple times to preview all data. 
try:
  name = next(previewIter)
except StopIteration:
  print("No more.")
else:
  x, y, w, h, img = loadData(name, 150)
  print(name)
  plt.imshow(img)

In [ ]:
# class MyCallback(tf.keras.callbacks.Callback):
#     def __init__(self, width, height):
#         super().__init__()
#         self.width = width
#         self.height = height
#     def on_epoch_begin(self, epoch, logs=None):
#         print(epoch)
#         sleep(1)
#         view(model, self.width, self.height, 5)
#         display.clear_output(wait=True)
#         display.display(plt.gcf())
# #         sleep(.01)

In [ ]:
def train(
    img_name, is_gray = False, SPF = 1.5, steps_per_epoch = 32, 
    canvas_size = (12, 5), 
    resolution = [150], 
    nn_width = [64], nn_depth = [3], 
    loss = [tf.keras.losses.mean_squared_error], 
):
    shape = []
    titles = []
    if len(resolution) > 1:
        shape.append(len(resolution))
        titles.append(('resolution', resolution))
    if len(nn_width) > 1:
        shape.append(len(nn_width))
        titles.append(('NN width', nn_width))
    if len(nn_depth) > 1:
        shape.append(len(nn_depth))
        titles.append(('NN depth', nn_depth))
    if len(loss) > 1:
        shape.append(len(loss))
        titles.append(('loss', ['L2' if x is tf.keras.losses.mean_squared_error else 'L1' for x in loss]))
    if len(shape) != 2:
        raise Exception('comparison limited to 2D, sorry')
    fig, axes = plt.subplots(*shape)
#     try:
#         iter(axes)
#     except TypeError:
#         axes = [axes]
    for i, ax in enumerate(axes[0, :]):
        ax.set_title(titles[1][0] + ' = ' + str(titles[1][1][i]))
    for i, ax in enumerate(axes[:, 0]):
        ax.set_ylabel(titles[0][0] + ' = ' + str(titles[0][1][i]))
    flat_axes = [x for t in axes for x in t]
    for ax in flat_axes:
        ax.tick_params(
            axis='both', which='both', 
            bottom=False, top=False, 
            labelbottom=False, 
            right=False, left=False, 
            labelleft=False, 
        )
    iterAxes = iter(flat_axes)
    max_w = 0
    max_h = 0
    for r in resolution:
        x, y, w, h, _ = loadData(img_name, r, is_gray)
        if w > max_w:
            max_w = w
            max_h = h
        for nw in nn_width:
            for nd in nn_depth:
                for l in loss:
                    model = MyModel(nw, nd, 1 if is_gray else 3)
                    model.compile(
                        optimizer='adam',
                        loss=l,
                    )
                    setups.append((model, x, y, w, h, next(iterAxes)))
    age = np.zeros((len(setups), ))
    epoch = np.zeros((len(setups), ), dtype=np.int32)
    next_render = 0
    render_i = 0
    while True:
        if next_render < np.sum(age):
            start = time()
            for model, x, y, w, h, ax in setups:
                output = model.predict(getRaster(max_w, max_h))
                ax.imshow(
                    np.reshape(output, (max_w, max_h, model.n_channels)), 
                    vmin=0, vmax=1, 
                )
            fig.set_size_inches(*canvas_size)
            fig.tight_layout()
            display.clear_output(wait=True)
            display.display(fig)
            next_render += SPF
            plt.savefig(f'../frames/{render_i}.jpg')
            render_i += 1
            print('Render overhead:', format((time() - start) / SPF, '.1%'))
            print('Epochs:')
            print(np.reshape(epoch, shape))
            model, x, y, w, h, ax = setups[-1]
            losses.append(model.evaluate(
                x, y, 
                batch_size = w * h, 
                steps = 1, 
                verbose = 0, 
            ))
            print('loss:', *losses[-5:], sep='\n')
        elected = np.argmin(age)
        model, x, y, w, h, ax = setups[elected]
        start = time()
        model.fit(
            x, y, 
            steps_per_epoch = steps_per_epoch, 
            epochs = 1, 
            verbose = 0, 
            batch_size = w * h, 
        )
        age[elected] += time() - start
        epoch[elected] += 1
    display.clear_output(wait=True)
    print("ok")

In [ ]:
setups = []
losses = []

train(
    'polyak et al.png', 
    is_gray = False, 
    SPF = 40, 
    canvas_size = (7, 5), 
    steps_per_epoch = 1, 
    resolution = [300, 300], 
    nn_width = [128, 128], 
    nn_depth = [6], 
#     nn_depth = [4, 8], 
#     loss = [tf.keras.losses.mean_squared_error, tf.keras.losses.mean_absolute_error], 
)

In [ ]:
np.save('../losses', losses)

In [ ]:
plt.plot(losses)

In [ ]:
def inspect(model, resolution = 150):
    activations = []
    activations.append(getRaster(resolution, resolution))
    for reluLayer in model.myLayers:
        activations.append(reluLayer(activations[-1]))
    activations.append(model.last(activations[-1]))
    fig, axes = plt.subplots(model.depth + 2, model.width)
    def draw(i, j, field, absolute = False):
        axes[i, j].imshow(
            np.reshape(field, (resolution, resolution)), 
            **({"vmin": 0, "vmax": 1} if absolute else {})
        )
        axes[i, j].axis('off')
    mid_col = model.width // 2
    draw(0, mid_col,     activations[0][:, 0])
    draw(0, mid_col + 1, activations[0][:, 1])
    for i in range(model.depth + 2):
        for j in range(model.width):
            if i in (0, model.depth + 1):
                axes[i, j].axis('off')
                continue
            draw(i, j, activations[i][:, j])
    for c in range(model.n_channels):
        draw(
            model.depth + 1, mid_col + c, 
            activations[model.depth + 1][:, c], absolute = True, 
        )
    fig.set_size_inches(model.width * 2, (model.depth + 2) * 2)

In [ ]:
inspect(model)

In [ ]:
inspect(cModels[1])

### Why does losses go up? 
1st experiment. Is `fit` deterministic? 

In [ ]:
x, y, w, h, _ = loadData('polyak et al.png', 300, False)
model = MyModel(128, 6, 3)
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.mean_squared_error,
)
model.fit(
    x, y, 
    steps_per_epoch = 1, 
    epochs = 1, 
#     verbose = 0, 
    batch_size = w * h, 
)
model_copy = model.copy()
model.evaluate(
    x, y, 
    batch_size = w * h, 
    steps = 1, 
)

In [ ]:
# Run multiple times

model = model_copy
model_copy = model.copy()
model.compile(
    optimizer='sgd',
    loss=tf.keras.losses.mean_squared_error,
)
model.fit(
    x, y, 
    steps_per_epoch = 1, 
    epochs = 1, 
#     verbose = 0, 
    batch_size = w * h, 
)
model.evaluate(
    x, y, 
    batch_size = w * h, 
    steps = 1, 
)

Yes. So they prolly need ideas from Blind Descend. 

In [ ]:
class CrutchGDScheduler:
    def __init__(self, acceleration = 1.1, verbose = True):
        self.lr = None
        self.acceleration = acceleration
        self.verbose = verbose

    def __call__(self, epoch, lr):
        if self.lr is None:
            self.lr = lr
            if self.verbose:
                print('Initial learning rate:', lr)
        print('learning rate adjustment:', self.lr / lr)
        return self.lr
    
    def succeed(self):
        self.lr *= self.acceleration
    
    def fail(self):
        self.lr *= .618

def crutchGD(model, x, y, w, h, lossFunc):
    sched = CrutchGDScheduler()
    callbacks = tf.keras.callbacks.LearningRateScheduler(sched)
    model.compile(
        optimizer='sgd',
        loss=lossFunc,
    )
    loss = np.inf
    while True:
        model_copy = model.copy()
        model.fit(
            x, y, 
            steps_per_epoch = 1, 
            epochs = 1, 
            verbose = 0, 
            batch_size = w * h, 
            callbacks = callbacks, 
        )
        new_loss = model.evaluate(
            x, y, 
            steps = 1, 
            verbose = 0, 
            batch_size = w * h, 
        )
        if loss < new_loss:
            model = model_copy
            model.compile(
                optimizer='sgd',
                loss=lossFunc,
            )
            sched.fail()
        else:
            loss = new_loss
            sched.succeed()
        yield model, loss

In [ ]:
def adam(model, x, y, w, h, lossFunc, steps_per_epoch):
    model.compile(
        optimizer='adam',
        loss=lossFunc,
    )
    while True:
        model.fit(
            x, y, 
            steps_per_epoch = steps_per_epoch, 
            epochs = 1, 
            verbose = 0, 
        )
        yield model

In [ ]:
ADAM = 'ADAM'
CRUTCH = 'CRUTCH'

def testCrutch(
    img_names, crutch_epoch = 3000, 
    is_gray = False, SPF = 1.5, steps_per_epoch = 32, 
    canvas_size = (10, 7), 
    resolution = 150, 
    nn_width = 64, nn_depth = 6, 
    lossFunc = tf.keras.losses.mean_squared_error, 
):
    fig, axes = plt.subplots(3, len(img_names))
    flat_axes = [x for t in axes for x in t]
    for ax in flat_axes:
        ax.tick_params(
            axis='both', which='both', 
            bottom=False, top=False, 
            labelbottom=False, 
            right=False, left=False, 
            labelleft=False, 
        )
    ground_truth = [
        loadData(img_name, resolution, is_gray) 
        for img_name in img_names
    ]
    models = [
        MyModel(nn_width, nn_depth, 1 if is_gray else 3) 
        for _ in img_names
    ]

    age = np.zeros((2, len(img_names)))
    epoch = np.zeros((2, len(img_names)), dtype=np.int32)
    next_render = 0
    render_i = 0

    phase = ADAM
    axes[0, 0].set_ylabel(ADAM)
    axes[1, 0].set_ylabel(ADAM)
    losses = [[[[], []] for _ in img_names] for _ in range(2)]
    trainers = [
        adam(model, x, y, w, h, lossFunc, steps_per_epoch)
        for model, (x, y, w, h, _) in zip(models, ground_truth)
    ]
    cModels = None
    try:
        while True:
            if phase is ADAM and np.min(epoch[0, :]) >= crutch_epoch:
                phase = CRUTCH
                cModels = [m.copy() for m in models]
                age[1, :] = age[0, :]
                axes[1, 0].set_ylabel(CRUTCH)
                cTrainers = [
                    crutchGD(model, x, y, w, h, lossFunc)
                    for model, (x, y, w, h, _) in zip(cModels, ground_truth)
                ]
            if next_render <= np.sum(age):
                next_render += SPF
                start = time()
                for i, (x, y, w, h, _) in enumerate(ground_truth):
                    if epoch[0, i] == 0:
                        continue
                    output = models[i].predict(getRaster(w, h))
                    reshaped = np.reshape(output, (w, h, model.n_channels))
                    axes[0, i].clear()
                    axes[0, i].imshow(reshaped, vmin=0, vmax=1)
                    if phase is CRUTCH:
                        if epoch[1, i] == 0:
                            continue
                        output = cModels[i].predict(getRaster(w, h))
                        reshaped = np.reshape(output, (w, h, model.n_channels))
                    axes[1, i].clear()
                    axes[1, i].imshow(reshaped, vmin=0, vmax=1)
                render_overhead = format((time() - start) / SPF, '.1%')
                start = time()
                for i, (x, y, w, h, _) in enumerate(ground_truth):
                    if epoch[0, i] == 0:
                        continue
                    loss_val = models[i].evaluate(
                        x, y, 
                        batch_size = w * h, 
                        steps = 1, 
                        verbose = 0, 
                    )
                    losses[0][i][0].append(render_i)
                    losses[0][i][1].append(loss_val)
                    if phase is CRUTCH:
                        if epoch[1, i] == 0:
                            continue
                        loss_val = cModels[i].evaluate(
                            x, y, 
                            batch_size = w * h, 
                            steps = 1, 
                            verbose = 0, 
                        )
                        losses[1][i][0].append(render_i)
                        losses[1][i][1].append(loss_val)
                eval_overhead = format((time() - start) / SPF, '.1%')
                start = time()
                for i, (x, y, w, h, _) in enumerate(ground_truth):
                    axes[2, i].clear()
                    axes[2, i].plot(losses[0][i][0], losses[0][i][1], label=ADAM)
                    axes[2, i].plot(losses[1][i][0], losses[1][i][1], label=CRUTCH)
                axes[2, -1].legend()
                axes[0, 0].set_ylabel(ADAM)
                axes[1, 0].set_ylabel(phase)
                axes[2, 0].set_ylabel('loss')
                pltLoss_overhead = format((time() - start) / SPF, '.1%')
                start = time()
                fig.set_size_inches(*canvas_size)
                fig.tight_layout()
                display.clear_output(wait=True)
                display.display(fig)
                print('Render overhead:', render_overhead)
                print('Evaluation overhead:', eval_overhead)
                print('plot loss overhead:', pltLoss_overhead)
                print('redraw overhead:', format((time() - start) / SPF, '.1%'))
                start = time()
                plt.savefig(f'../frames/{render_i}.jpg')
                print('save fig overhead:', format((time() - start) / SPF, '.1%'))
                render_i += 1
                print('Epochs (the two rows are not comparable):')
                print(epoch)
            if phase is ADAM:
                col = age[0, :].argmin()
                row = 0
            else:
                row, col = np.unravel_index(age.argmin(), age.shape)
            start = time()
            if row == 0:
                models[col] = next(trainers[col])
            else:
                cModels[col], _ = next(cTrainers[col])
            age[row, col] += time() - start
            epoch[row, col] += 1
    except KeyboardInterrupt:
        return models, cModels, ground_truth, losses

In [ ]:
models, cModels, ground_truth, losses = testCrutch(
    ['sinGAN_2.png', 'sinGAN_1.jpg', 'shoob_3.jpg'], 
    crutch_epoch = 30000, 
    nn_width = 64, 
    canvas_size = (10 * .7, 7 * .7), SPF = 20, 
)

In [ ]:
with open('../archive/ground_truth_losses.pickle', 'wb') as f:
    pickle.dump([ground_truth, losses], f)

In [ ]:
models[2].save('../archive/model')

In [ ]:
with open('../archive/crutch_compare/ground_truth_losses.pickle', 'rb') as f:
    ground_truth_demo, losses_demo = pickle.load(f)

START = 20
_START = 0

fig, axes = plt.subplots(1, 3)
for i, (x, y, w, h, _) in enumerate(ground_truth_demo):
    axes[i].plot(losses_demo[0][i][0][START:], losses_demo[0][i][1][START:], label=ADAM)
    axes[i].plot(losses_demo[1][i][0][_START:], losses_demo[1][i][1][_START:], label=CRUTCH)
axes[-1].legend()
fig.set_size_inches(10, 4)

Okay. it's also cheap to evaluate on the whole training set. Compare with RejectionAdam? 

In [ ]:
def rejectableAdam(
    model, x, y, w, h, lossFunc, steps_per_epoch, epochs_per_eval, 
    crutchAsCure = False, 
):
    model.compile(
        optimizer='adam',
        loss=lossFunc,
    )
    loss = np.inf
    while True:
        model_copy = model.copy()
        start = perf_counter()
        for _ in range(epochs_per_eval):
            model.fit(
                x, y, 
                steps_per_epoch = steps_per_epoch, 
                epochs = 1, 
                verbose = 0, 
            )
        fit_time = perf_counter() - start
        start = perf_counter()
        new_loss = model.evaluate(
            x, y, 
            steps = 1, 
            verbose = 0, 
            batch_size = w * h, 
        )
        eval_time = perf_counter() - start
        if loss < new_loss:
            model = model_copy
            if crutchAsCure:
                sched = CrutchGDScheduler(verbose = False)
                callbacks = tf.keras.callbacks.LearningRateScheduler(sched)
                model.compile(
                    optimizer='sgd',
                    loss=lossFunc,
                )
                while True:
                    model_copy = model.copy()
                    model.fit(
                        x, y, 
                        steps_per_epoch = 1, 
                        epochs = 1, 
                        verbose = 0, 
                        batch_size = w * h, 
                        callbacks = callbacks, 
                    )
                    new_loss = model.evaluate(
                        x, y, 
                        steps = 1, 
                        verbose = 0, 
                        batch_size = w * h, 
                    )
                    if loss < new_loss:
                        model = model_copy
                        model.compile(
                            optimizer='sgd',
                            loss=lossFunc,
                        )
                        sched.fail()
                    else:
                        break
            model.compile(
                optimizer='sgd',
                loss=lossFunc,
            )
        else:
            loss = new_loss
#         print('IN-TRAIN eval overhead:', eval_time / fit_time)
        yield model, loss

In [ ]:
def testRejectables(
    img_names, 
    is_gray = False, SPF = 1.5, steps_per_epoch = 32, 
    canvas_size = (10, 7), 
    resolution = 150, 
    nn_width = 64, nn_depth = 6, 
    lossFunc = tf.keras.losses.mean_squared_error, 
    epochs_per_eval = 1, 
):
    fig, axes = plt.subplots(3, len(img_names))
    flat_axes = [x for t in axes for x in t]
    for ax in flat_axes:
        ax.tick_params(
            axis='both', which='both', 
            bottom=False, top=False, 
            labelbottom=False, 
            right=False, left=False, 
            labelleft=False, 
        )
    ground_truth = [
        loadData(img_name, resolution, is_gray) 
        for img_name in img_names
    ]
    models = [[], []]
    for _ in img_names:
        model = MyModel(nn_width, nn_depth, 1 if is_gray else 3)
        model.build()
        models[0].append(model)
        models[1].append(model.copy())

    age = np.zeros((2, len(img_names)))
    epoch = np.zeros((2, len(img_names)), dtype=np.int32)
    next_render = 0
    render_i = 0

    losses = [[[[], []] for _ in img_names] for _ in range(2)]
    trainers = [[], []]
    for row, clutch_as_cure in enumerate((False, True)):
        for col, (x, y, w, h, _) in enumerate(ground_truth):
            trainers[row].append(rejectableAdam(
                models[row][col], x, y, w, h, lossFunc, 
                steps_per_epoch, epochs_per_eval, 
                clutch_as_cure, 
            ))
    try:
        while True:
            if next_render <= np.sum(age):
                next_render += SPF
                start = perf_counter()
                for col, (x, y, w, h, _) in enumerate(ground_truth):
                    for row in (0, 1):
                        if epoch[row, col] == 0:
                            continue
                        model = models[row][col]
                        output = model.predict(getRaster(w, h))
                        reshaped = np.reshape(output, (w, h, model.n_channels))
                        axes[row, col].clear()
                        axes[row, col].imshow(reshaped, vmin=0, vmax=1)
                render_overhead = format((perf_counter() - start) / SPF, '.1%')
                start = perf_counter()
                for row in (0, 1):
                    for col, (x, y, w, h, _) in enumerate(ground_truth):
                        if epoch[row, col] == 0:
                            continue
                        loss_val = models[row][col].evaluate(
                            x, y, 
                            batch_size = w * h, 
                            steps = 1, 
                            verbose = 0, 
                        )
                        losses[row][col][0].append(render_i)
                        losses[row][col][1].append(loss_val)
                eval_overhead = format((perf_counter() - start) / SPF, '.1%')
                start = perf_counter()
                for col, (x, y, w, h, _) in enumerate(ground_truth):
                    axes[-1, col].clear()
                    for row, label in enumerate(['RejAdam', 'Crutch']):
                        axes[-1, col].plot(losses[row][col][0], losses[row][col][1], label=label)
                axes[-1, -1].legend()
                axes[0, 0].set_ylabel('RejAdam')
                axes[1, 0].set_ylabel('Crutch')
                axes[2, 0].set_ylabel('loss')
                pltLoss_overhead = format((perf_counter() - start) / SPF, '.1%')
                start = perf_counter()
                fig.set_size_inches(*canvas_size)
                fig.tight_layout()
                display.clear_output(wait=True)
                display.display(fig)
                print('Render overhead:', render_overhead)
                print('Evaluation overhead:', eval_overhead)
                print('plot loss overhead:', pltLoss_overhead)
                print('redraw overhead:', format((perf_counter() - start) / SPF, '.1%'))
                start = perf_counter()
                plt.savefig(f'../frames/{render_i}.jpg')
                print('save fig overhead:', format((perf_counter() - start) / SPF, '.1%'))
                render_i += 1
                print('Epochs (the two rows are not comparable):')
                print(epoch)
            row, col = np.unravel_index(age.argmin(), age.shape)
            start = perf_counter()
            models[row][col], _ = next(trainers[row][col])
            age[row, col] += perf_counter() - start
            epoch[row, col] += 1
    except KeyboardInterrupt:
        return models, ground_truth, losses

In [ ]:
models, ground_truth, losses = testRejectables(
    ['sinGAN_2.png', 'sinGAN_1.jpg', 'shoob_3.jpg'], 
    nn_width = 64, 
    canvas_size = (10 * .7, 7 * .7), SPF = 20, 
    steps_per_epoch = 128, epochs_per_eval = 8, 
)